In [3]:
import pandas as pd
data_30_sentences = pd.read_csv(r'C:\Users\erich\Desktop\DS_project\data\data_30_sentences.csv')

In [11]:
pip install daal==2021.4.0

  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [ ]:
pip install numpy==1.21.2

In [10]:
!pip uninstall -y numpy tensorflow
!pip install numpy tensorflow

Found existing installation: numpy 1.25.1
Uninstalling numpy-1.25.1:
  Successfully uninstalled numpy-1.25.1
Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0
  Using cached tensorflow-2.13.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

lemmatizer = WordNetLemmatizer()

def clean_data(df):
    df = df[df['HTML_Content'].isin(['Explicit', 'General Audiences', 'Mature', 'Not Rated', 'Teen And Up Audiences'])]

    df = df.dropna(subset=['TXT_Content'])

    def clean_text(text):

        if not isinstance(text, str):
            text = str(text)

        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        tokenized_text = word_tokenize(text)
        cleaned_text = [lemmatizer.lemmatize(word) for word in tokenized_text if word not in set(stopwords.words('english'))]

        return ' '.join(cleaned_text)

    df['TXT_Content'] = df['TXT_Content'].apply(clean_text)

    return df

data_30_sentences = clean_data(data_30_sentences)

print(data_30_sentences)

               HTML_Content                                        TXT_Content
0     Teen And Up Audiences  huckleberry finn stood front old wooden door m...
1     Teen And Up Audiences  right huck trying figure every sort way could ...
2         General Audiences  three buried mother christmas eve thing huck c...
3     Teen And Up Audiences  air hot brow wet mind exhausted st petersburg ...
4         General Audiences  episode genre al chaos chaos space time vortex...
...                     ...                                                ...
5993  Teen And Up Audiences  seen news pat asked robin chance hang coat mor...
5994  Teen And Up Audiences  free next friday night murphy smiled phone rob...
5995                 Mature  turned walked away wan na say come baby give w...
5996      General Audiences  perfect word tended apply liberally one often ...
5997                 Mature  heaven fact unlike earth pro con pro include t...

[5998 rows x 2 columns]


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, TimeDistributed, Flatten, Attention
from keras.models import Sequential
from keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import time


# Pre-processing
max_length = 300
embedding_dim = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_30_sentences['TXT_Content'])
sequences = tokenizer.texts_to_sequences(data_30_sentences['TXT_Content'])
word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=max_length)

labels = to_categorical(np.asarray(LabelEncoder().fit_transform(data_30_sentences['HTML_Content'])))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Word Embedding
glove_input_file = 'C:\\Users\\erich\\Desktop\\DS_project\\code\\glove.6B\\glove.6B.100d.txt'
word2vec_output_file = 'C:\\Users\\erich\\Desktop\\DS_project\\code\\glove.6B\\glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

word2vec = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]


C:\Users\erich\AppData\Local\Temp\ipykernel_28200\475069023.py:34: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [14]:
print("Example of a sequence:", sequences[0])
print("Example of a label:", labels[0])  
print("Embedding matrix:", embedding_matrix)

for word, i in list(word_index.items())[:5]:
    print(f"Word: {word}, Vector: {embedding_matrix[i]}")


Example of a sequence: [3822, 3154, 255, 141, 87, 1367, 52, 201, 4911, 1155, 3253, 4658, 210, 450, 2810, 6, 322, 18, 66, 251, 176, 497, 5432, 366, 5, 1215, 4659, 895, 10855, 268, 296, 2901, 16, 81, 461, 111, 156, 1768, 296, 72, 2666, 134, 30, 756, 4, 314, 16, 371, 720, 41, 3878, 6558, 27192, 273, 3536, 10278, 81, 137, 680, 7943, 1135, 3536, 3376, 985, 6385, 11471, 714, 279, 25, 48, 5920, 499, 1225, 10855, 116, 16963, 856, 281, 79, 3878, 10279, 1070, 322, 18, 1870, 84, 305, 205, 1628, 4425, 52, 357, 33, 564, 22, 6, 3536, 8, 1785, 14051, 3289, 4053, 771, 1753, 714, 3496, 3673, 714, 7944, 3253, 90, 5433, 383, 71, 579, 8, 168, 137, 579, 322, 18, 207, 4, 553, 1, 17, 318, 4, 553, 1, 17, 318, 236, 25, 296, 55, 579, 765, 163, 439, 669, 560, 30, 3377, 2835, 55, 276, 49, 253, 93, 378, 4290, 13, 2066, 3, 10, 182, 264, 79, 119, 2106, 2489, 19, 33, 3822, 34, 4660, 49, 79, 40, 378, 13, 2066, 76, 34, 656, 724, 21985, 2066, 49, 714, 650, 7, 1455, 15, 10856, 118, 774, 246, 6, 24, 78, 79, 280, 178, 12, 

In [15]:
from keras.layers import Input
from keras.models import Model
from sklearn.metrics import classification_report
import keras.backend as K

# Model structure
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

inputs = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(inputs)
lstm = LSTM(100, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
preds = Dense(labels.shape[1], activation='softmax')(lstm)

model = Model(inputs=inputs, outputs=preds)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', Precision(), Recall()])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[EarlyStopping(monitor='val_loss', patience=3)])


y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test_argmax = np.argmax(y_test, axis=1)

# Calculate metrics
print(classification_report(y_test_argmax, y_pred))

K.clear_session()


Epoch 1/10
75/75 [==============================] - 133s 2s/step - loss: 1.5454 - accuracy: 0.3218 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.5563 - val_accuracy: 0.3108 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 2/10
75/75 [==============================] - 111s 1s/step - loss: 1.5218 - accuracy: 0.3289 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.5529 - val_accuracy: 0.3050 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 3/10
75/75 [==============================] - 116s 2s/step - loss: 1.5075 - accuracy: 0.3378 - precision_4: 0.7500 - recall_4: 6.2526e-04 - val_loss: 1.5555 - val_accuracy: 0.3108 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 4/10
75/75 [==============================] - 120s 2s/step - loss: 1.4903 - accuracy: 0.3466 - precision_4: 0.5238 - recall_4: 0.0023 - val_loss: 1.5571 - val_accuracy: 0.3100 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 5/10
38/38 [======